In [13]:
import math
from typing import Optional, Tuple
import os, base64

from smolagents import tool

In [14]:
from dotenv import load_dotenv
load_dotenv()

True

## Setting Telemetry

In [15]:
LANGFUSE_PUBLIC_KEY= os.getenv('LANGFUSE_PUBLIC_KEY')
LANGFUSE_SECRET_KEY= os.getenv('LANGFUSE_SECRET_KEY')
LANGFUSE_AUTH=base64.b64encode(f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}".encode()).decode()


os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "http://localhost:3000/api/public/otel" # EU data region
# os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://us.cloud.langfuse.com/api/public/otel" # US data region
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"

In [16]:
from opentelemetry.sdk.trace import TracerProvider
from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

trace_provider = TracerProvider()
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter()))

SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)

## Agent Creation

In [17]:
@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750,
) -> float:
    """
    Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750 km/h for typical cargo planes)

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return math.radians(degrees)
    
    # Extract coordinates
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371.0

    # Calculate great-circle distance using the haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # Add 10% to account for non-direct routes and air traffic controls
    actual_distance = distance * 1.1

    # Calculate flight time
    # Add 1 hour for takeoff and landing procedures
    flight_time = (actual_distance / cruising_speed_kmh) + 1.0

    # Format the results
    return round(flight_time, 2)
  

In [18]:
print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))

22.82


In [19]:
import os
from PIL import Image
from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel, VisitWebpageTool


model = HfApiModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")

In [ ]:
task = """Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.
Also give me some supercar factories with the same cargo plane transfer time."""

In [ ]:
agent = CodeAgent(
    model=model,
    tools=[DuckDuckGoSearchTool(), VisitWebpageTool(), calculate_cargo_travel_time],
    additional_authorized_imports=["pandas"],
    max_steps=20,
)

In [20]:
result = agent.run(task)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_locations = web_search(query="Batman filming locations")                                                  
  print(batman_locations)                                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Where The Batman Filmed (All Gotham Real-Life Locations) - Screen 
Rant](https://screenrant.com/where-was-the-batman-movie-filmed/)
The London location known as Two Temple Place served as another The Batman filming location. Two Temple Place was 
the first building seen in the movie, in fact, as it was the home of Gotham's Mayor Don Mitchell. Matt Reeves began
The Batman by showing the location through Riddler's eyes. Two Temple Place then became a major location for the 
...

[The Batman | Film Locations](https://www.movie-locations.com/movies/b/The-Batman-2022.php)
The Batman location: the subterranean entrance to 'Wayne Tower': Holborn tram tunnel, Holborn It's accessed via a 
subterranean passage, which is the old Kingsway Tram Tunnel , disused since trams were discontinued in London in 
1952, smack in the middle of busy Southampton Row just south of Theobald's Road near Holborn tube station.

[Where was The Batman filmed? ALL the Filming Locations in Chicago & The 
UK](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
The Batman is the newest addition to the Caped Crusader films franchise. Matt Reeves, who also directed the last 
two films in the Planet of the Apes movie series, has made a more noir, detective-driven version of the story. The 
Batman was primarily filmed in the United Kingdom.Most of the recognizable buildings of this new version of Gotham 
City are located in Liverpool, although Chicago was a ...

[12 Batman Movie Locations You Can Visit! - Leisure Asia 
Global](https://www.travelandleisureasia.com/global/destinations/batman-movie-locations-you-can-visit/)
Image: Courtesy Rob Farrow / Two, Temple Place, London/ Creative Commons Attribution-Share Alike 2.0 Generic/ 
Wikimedia Commons The stunning Victorian mansion in London is another officially IMDb listed filming location for 
Matt Reeves' The Batman.The building is as gorgeous as it can be and retains the classic European charm till date. 
The venue has been featured in several other noted ...

[10 Iconic Batman Filming Locations You Can Actually Visit - 
WhatCulture](https://whatculture.com/film/10-iconic-batman-filming-locations-you-can-actually-visit)
Every night during the filming of The Dark Knight, South Wacker Drive was barricaded off from 7 pm to 7 am to 
accommodate the movie. The street itself is an underground tunnel of sorts.

[The Batman Filming Locations: Where was Matt Reeves Directorial 
Filmed?](https://www.theartistree.fm/entertainment/449883/the-batman-filming-locations-where-was-matt-reeves-direct
orial-filmed/)
Matt Reeves' The Batman was filmed in many places around the globe to create a new version of Gotham and used 
real-life locations to capture the right look. This new take on the Dark Knight introduced audiences to Robert 
Pattinson's Batman and the environment he lives in. Along with bringing characters like Penguin, Catwoman, and 
others from DC Comics to life, the movie truly made Gotham ...

[These Are the Cities Standing in for Gotham in 'The Batman' - 
Distractify](https://www.distractify.com/p/the-batman-filming-locations)
The latest film centered around the Caped Crusader, The Batman, is finally in theaters.One of the key elements of 
any good Batman story is the Gotham City setting. Over the years, various directors have chosen different cities to
stand in for the crime-riddled location, but where did Matt Reeves set his epic Batman tale?

[The Batman - filming locations - SCEEN IT](https://www.sceen-it.com/movie/1213/The-Batman)
All filming locations of The Batman from 2022 are listed below. STORY In his second year of fighting crime, Batman 
uncovers corruption in Gotham City that connects to his own family while facing a serial killer known as the 
Riddler.

[Where Was The Batman Filmed? 2022 Movie Filming 
Locations](https://thecinemaholic.com/where-was-the-batman-filmed/)
The Batman Filming Locations 'The Batman' was filmed in various parts of England a

[Step 0: Duration 9.88 seconds| Input tokens: 2,377 | Output tokens: 160]

...

[Step 2: Duration 14.31 seconds| Input tokens: 12,843 | Output tokens: 1,101]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Coordinates for supercar factories                                                                             
  supercar_factories = {                                                                                           
      "McLaren, Woking, England": (51.3002, -0.5977),                                                              
      "McLaren, Sheffield, England": (53.3811, -1.4709),                                                           
      "McLaren, Bicester, England": (51.6538, -1.2579),                                                            
      "Ferrari, Maranello, Italy": (44.5646, 10.9941),                                                             
      "Acura, Maryland, USA": (39.0458, -76.6413)                                                                  
  }                                                                                                                
                                                                                                                   
  # Calculate travel time for each supercar factory                                                                
  supercar_travel_times = {}                                                                                       
  for location, coords in supercar_factories.items():                                                              
      travel_time = calculate_cargo_travel_time(origin_coords=coords, destination_coords=gotham_coords)            
      supercar_travel_times[location] = travel_time                                                                
                                                                                                                   
  # Create a DataFrame for supercar factories                                                                      
  supercar_df = pd.DataFrame(list(supercar_travel_times.items()), columns=['Location', 'Travel Time (hours)'])     
                                                                                                                   
  # Filter supercar factories with the same travel time as Batman filming locations                                
  batman_travel_times_set = set(batman_travel_times.values())                                                      
  filtered_supercar_df = supercar_df[supercar_df['Travel Time (hours)'].isin(batman_travel_times_set)]             
                                                                                                                   
  # Print Batman filming locations DataFrame                                                                       
  print("Batman Filming Locations:")                                                                               
  print(batman_df)                                                                                                 
                                                                                                                   
  # Print filtered supercar factories DataFrame                                                                    
  print("\nSupercar Factories with the Same Travel Time to Gotham:")                                               
  print(filtered_supercar_df)                                                                                      
                                                                                                                   
  # Combine both DataFrames                                                                                        
  combined_df = pd.concat([batman_df, filtered_supercar_df], ignore_index=True)                                    
                                                                                                                   
  # Print combined DataFrame                             

Execution logs:
Batman Filming Locations:
             Location  Travel Time (hours)
0  Liverpool, England                 8.81
1     London, England                 9.17
2    Watford, England                 9.14
3    Bedford, England                 9.11
4   Glasgow, Scotland                 8.60
5  Hartwood, Scotland                 8.65
6   Chicago, Illinois                 2.68

Supercar Factories with the Same Travel Time to Gotham:
Empty DataFrame
Columns: [Location, Travel Time (hours)]
Index: []

Combined Batman Filming Locations and Supercar Factories:
             Location  Travel Time (hours)
0  Liverpool, England                 8.81
1     London, England                 9.17
2    Watford, England                 9.14
3    Bedford, England                 9.11
4   Glasgow, Scotland                 8.60
5  Hartwood, Scotland                 8.65
6   Chicago, Illinois                 2.68

Out - Final answer:              Location  Travel Time (hours)
0  Liverpool, England                 8.81
1     London, England                 9.17
2    Watford, England                 9.14
3    Bedford, England                 9.11
4   Glasgow, Scotland                 8.60
5  Hartwood, Scotland                 8.65
6   Chicago, Illinois                 2.68

[Step 3: Duration 12.74 seconds| Input tokens: 21,955 | Output tokens: 2,117]

In [21]:
agent.planning_interval = 4

detailed_report = agent.run(f"""
You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

{task}
""")

print(detailed_report)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're an expert analyst. You make comprehensive reports after visiting many websites.                          │
│ Don't hesitate to search for many queries at once in a for loop.                                                │
│ For each data point that you find, visit the source url to confirm numbers.                                     │
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here is the plan of action that I will follow to solve the task:
                ```
                1. Perform a web search to find a comprehensive list of Batman filming locations worldwide.
2. For each filming location found, visit the source URL to confirm the coordinates.
3. Perform a web search to find a comprehensive list of supercar factories worldwide.
4. For each supercar factory found, visit the source URL to confirm the coordinates.
5. Calculate the cargo plane transfer time from Gotham to each Batman filming location using the provided tool.
6. Calculate the cargo plane transfer time from Gotham to each supercar factory using the provided tool.
7. Organize the collected data into a pandas dataframe with columns for location name, coordinates, distance from 
Gotham, and estimated cargo plane transfer time.
8. Finalize the pandas dataframe to include both Batman filming locations and supercar factories with the same 
cargo plane transfer time.
9. Provide the final answer as the pandas dataframe.


                ```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_filming_locations = web_search(query="Batman filming locations worldwide")                                
  print(batman_filming_locations)                                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Where The Batman Filmed (All Gotham Real-Life Locations) - Screen 
Rant](https://screenrant.com/where-was-the-batman-movie-filmed/)
Warning: Spoilers for The Batman!. Matt Reeves' The Batman filmed all over the world to bring a new version of 
Gotham to life and went to some prominent real-life locations to achieve the right look. Another new interpretation
of the Dark Knight in live-action meant introducing audiences to Robert Pattinson's Batman and the world that he 
inhabits.

[Where was The Batman filmed? ALL the Filming Locations in Chicago & The 
UK](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
The Batman is the newest addition to the Caped Crusader films franchise. Matt Reeves, who also directed the last 
two films in the Planet of the Apes movie series, has made a more noir, detective-driven version of the story. The 
Batman was primarily filmed in the United Kingdom.Most of the recognizable buildings of this new version of Gotham 
City are located in Liverpool, although Chicago was a ...

[The Batman | Film Locations - Worldwide Guide To Movie 
Locations](https://www.movie-locations.com/movies/b/The-Batman-2022.php)
The Batman location: the subterranean entrance to 'Wayne Tower': Holborn tram tunnel, Holborn It's accessed via a 
subterranean passage, which is the old Kingsway Tram Tunnel , disused since trams were discontinued in London in 
1952, smack in the middle of busy Southampton Row just south of Theobald's Road near Holborn tube station.

[Where Was The Batman Filmed? | iFILMthings](https://ifilmthings.com/where-was-the-batman-filmed/)
The Batman movie was filmed in various locations around the world, including London and Liverpool in England, 
Chicago in the United States, and parts of Northern Ireland. In London, filming took place at iconic locations such
as Trafalgar Square where a scene featuring Robert Pattinson's Bruce Wayne aboard his Batcycle was filmed, and 
around ...

[Batman | 1989 - Worldwide Guide To Movie Locations](https://movie-locations.com/movies/b/Batman-1989.php)
Film locations for Tim Burton's Batman in London and Southeast England: Knebworth House, Knebworth, Hertfordshire 
('Wayne Manor' exterior); Hatfield House, Hatfield, Hertfordshire ('Wayne Manor' interior); Pinewood Studio, 
Buckinghamshire (studio); Ladies' Smoking Room, St Pancras Renaissance Hotel, Euston Road, NW1 (office of Gotham 
Globe); Little Barford Power Station, St Neots ...

[12 Batman movie locations around the world you can actually 
visit](https://www.lifestyleasia.com/sg/entertainment/the-batman-movie-locations-around-the-world-you-can-visit/)
12 real-world Batman movie locations that you can visit now St. George's Hall, Liverpool. Image: Courtesy 
Superchilum, CC BY-SA 4.0, via Wikimedia Commons. One of the many IMDb listed filming locations for Reeves' The 
Batman is the iconic St. George's Hall in Liverpool, England. Though we have seen different variations of Gotham 
City ...

[Where Was The Batman Filmed? 2022 Movie Filming 
Locations](https://thecinemaholic.com/where-was-the-batman-filmed/)
Fans worldwide are highly curious to find out about the exciting places where 'The Batman' was filmed. Without 
further delay, here are the answers to everyone's queries. Let's begin! The Batman Filming Locations 'The Batman' 
was filmed in various parts of England and Scotland, specifically in Liverpool, London, Watford, Bedford ...

[The Batman Filming Locations: Where was Matt Reeves Directorial 
Filmed?](https://www.theartistree.fm/entertainment/449883/the-batman-filming-locations-where-was-matt-reeves-direct
orial-filmed/)
Matt Reeves' The Batman was filmed in many places around the globe to create a new version of Gotham and used 
real-life locations to capture the right look. This new take on the Dark Knight introduced audiences to Robert 
Pattinson's Batman and the environment he lives in. Along with bringing characters like Penguin, Catwoman, and 
o

...

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
                                                                                                                   
  # Batman filming locations with confirmed coordinates                                                            
  batman_locations = {                                                                                             
      "Major Mitchell's House": {"coordinates": (51.5138, -0.0985)},                                               
      "Wellington's Column and Steble Fountain": {"coordinates": (53.4084, -2.9916)},                              
      "St George's Hall": {"coordinates": (53.4084, -2.9916)},                                                     
      "Liverpool Anglican Cathedral": {"coordinates": (53.4090, -2.9915)},                                         
      "Chicago": {"coordinates": (41.8781, -87.6298)}                                                              
  }                                                                                                                
                                                                                                                   
  # Gotham's coordinates                                                                                           
  gotham_coords = (40.7128, -74.0060)                                                                              
                                                                                                                   
  # Calculate cargo plane transfer times for Batman filming locations                                              
  batman_transfer_times = {}                                                                                       
  for location, info in batman_locations.items():                                                                  
      transfer_time = calculate_cargo_travel_time(origin_coords=gotham_coords,                                     
  destination_coords=info['coordinates'])                                                                          
      batman_transfer_times[location] = transfer_time                                                              
                                                                                                                   
  # Perform a web search to find a comprehensive list of supercar factories worldwide                              
  supercar_factories = web_search(query="supercar factories worldwide")                                            
  print(supercar_factories)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Top 10 Supercar Manufacturers In The World Today - Ape To 
Gentleman](https://www.apetogentleman.com/top-10-supercar-brands/)
The supercar brands that, through engineering or design, have pushed the car industry forward and are responsible 
for most modern-day technologies.

[10 Best Supercar Brands in the World - ExperienceFerrari](https://www.experienceferrari.com/best-supercar-brands/)
Learn the top 10 supercar brands worldwide, from Ferrari to Bugatti. Explore speed, innovation, and luxury in 
automotive excellence.

[The Best Supercar Manufacturers In The World (Ranked) - 
MSN](https://www.msn.com/en-us/autos/performance-cars/the-best-supercar-manufacturers-in-the-world-ranked/ss-AA1vne
Re)
The story of supercar manufacturing is one of passion and endless innovation. Besides their super stylish look, 
these are high-performance automobiles that bring along a thrill while driving. Here ...

[Top 10 Super Car Manufacturers - 
Profestyle](https://profestyle.com/top-10-super-car-manufacturers-in-the-world-elite-picks/)
Each of these brands has a story, a soul, and a heart that beats fast. Frequently Asked Questions Of Top 10 Super 
Car Manufacturers In The World: Elite Picks! Who Leads Supercar Manufacturing? Italian powerhouse Ferrari 
frequently dominates the supercar manufacturing sector with its blend of luxury, performance, and timeless design.

[The Best Supercar Brands in the World - CarLogos.org](https://www.carlogos.org/supercar-brands/)
The Best Supercar Brands in the World Bugatti, SSC, Koenigsegg, and Hennessey are definitely among the most 
well-known and respected in the supercar industry. Each of these brands has earned a reputation for producing some 
of the most powerful, technologically advanced, and exclusive vehicles in the world.

[World supercars, best world supercars manufacturing industries, deluxe 
...](https://www.supercarsmanufacturing.com/world_supercars_manufacturing_sport_automotive_industries.htm)
BEST WORLD SUPERCARS MANUFACTURERS list of the best global sport supercars manufacturing industries for 2021 
combining high performance, hybrid sport automobiles and street legal homologation, certified industrial automotive
manufacturers as Porsche AG, Ferrari S.p.A, McLaren, Maserati, Bugatti, Sly Garage Factory, Lamborghini, Pagani 
offer the best automotive supercars technology and customer ...

[17 Best Supercar Brands (#10 Is a Beast) - Auto Care Aids](https://autocareaids.com/best-supercar-brands/)
All McLaren supercars are produced in the company's manufacturing plants using an in-house workforce and equipment.
McLaren Automotive has won several Formula One world championships. Some of the brand's best models include the 
no-windscreen Elva roadster, Speedtail, 720S, and Artura. McLaren is basically Britain's finest supercar ...

[Fast, Flashy, And Pricey: The Top Supercar Manufacturers - 
TheRichest](https://www.therichest.com/business/life-in-the-fast-lane-the-top-supercar-manufacturers-in-the-world/)
The world of supercars is an ever changing environment, with the newest cutting edge automotive technologies 
battling for a slice of a very niche market. Of course, amidst the competition there are still some supercar 
manufacturers that stand out above the rest. These are the companies that are revered and known by car enthusiasts 
around the world.

[The Top Super Car Manufacturers: Unveiling the Elite Brands in the 
...](https://rentavettephoenix.com/super-car-manufacturers-the-ultimate-guide/)
Super car manufacturers: Explore the world of luxury and high-performance vehicles from top brands. Get insights 
into the latest innovations and designs.

[15 Best Supercar Brands 2019 - Top Supercar Brands to 
Know](https://www.esquiremag.ph/wheels/cars/best-supercar-brands-a2336-20190613-lfrm)
Since the first supercar, Lamborghini's Miura, was produced, plenty of brands have popped up to serve the 
ultra-luxury segment. As a guide, we've compiled a list of the b

[Step 3: Duration 14.17 seconds| Input tokens: 44,943 | Output tokens: 1,688]

[Step 4: Duration 0.74 seconds| Input tokens: 70,553 | Output tokens: 2,389]

HfHubHTTPError: 402 Client Error: Payment Required for url: https://router.huggingface.co/together/v1/chat/completions (Request ID: Root=1-67c5cfbc-1c17a7a041be26dc12dbf27a;148d02f1-fc83-44a4-9de4-301c8fea101c)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly allowance.

In [24]:
agent.visualize()

CodeAgent | Qwen/Qwen2.5-Coder-32B-Instruct
├── ✅ Authorized imports: ['pandas']
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name                        ┃ Description                           ┃ Arguments                             ┃
    ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ web_search                  │ Performs a duckduckgo web search      │ query (`string`): The search query to │
    │                             │ based on your query (think a Google   │ perform.                              │
    │                             │ search) then returns the top search   │                                       │
    │                             │ results.                              │                                       │
    │ visit_webpage               │ Visits a webpage at the given url and │ url (`string`): The url of the        │
    │                             │ reads its content as a markdown       │ webpage to visit.                     │
    │                             │ string. Use this to browse webpages.  │                                       │
    │ calculate_cargo_travel_time │ Calculate the travel time for a cargo │ origin_coords (`array`): Tuple of     │
    │                             │ plane between two points on Earth     │ (latitude, longitude) for the         │
    │                             │ using great-circle distance.          │ starting point                        │
    │                             │                                       │ destination_coords (`array`): Tuple   │
    │                             │                                       │ of (latitude, longitude) for the      │
    │                             │                                       │ destination                           │
    │                             │                                       │ cruising_speed_kmh (`number`):        │
    │                             │                                       │ Optional cruising speed in km/h       │
    │                             │                                       │ (defaults to 750 km/h for typical     │
    │                             │                                       │ cargo planes)                         │
    │ final_answer                │ Provides a final answer to the given  │ answer (`any`): The final answer to   │
    │                             │ problem.                              │ the problem                           │
    └─────────────────────────────┴───────────────────────────────────────┴───────────────────────────────────────┘

In [29]:
agent.write_memory_to_messages()

[{'role': <MessageRole.SYSTEM: 'system'>,
  'content': [{'type': 'text',
    'text': 'You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.\nTo do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.\nTo solve the task, you must plan forward to proceed in a series of steps, in a cycle of \'Thought:\', \'Code:\', and \'Observation:\' sequences.\n\nAt each step, in the \'Thought:\' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.\nThen in the \'Code:\' sequence, you should write the code in simple Python. The code sequence must end with \'<end_code>\' sequence.\nDuring each intermediate step, you can use \'print()\' to save whatever important information you will then need.\nThese print outputs will then appear in the \'Observation:\' field, which will be available as input for the next step.

In [30]:
agent.authorized_imports

['itertools',
 'time',
 'queue',
 'datetime',
 'pandas',
 'statistics',
 're',
 'unicodedata',
 'collections',
 'math',
 'stat',
 'random']

In [32]:
agent.python_executor.state

{'_print_outputs': PrintContainer(## Search Results
 
 [Top 10 Supercar Manufacturers In The World Today - Ape To Gentleman](https://www.apetogentleman.com/top-10-supercar-brands/)
 The supercar brands that, through engineering or design, have pushed the car industry forward and are responsible for most modern-day technologies.
 
 [10 Best Supercar Brands in the World - ExperienceFerrari](https://www.experienceferrari.com/best-supercar-brands/)
 Learn the top 10 supercar brands worldwide, from Ferrari to Bugatti. Explore speed, innovation, and luxury in automotive excellence.
 
 [The Best Supercar Manufacturers In The World (Ranked) - MSN](https://www.msn.com/en-us/autos/performance-cars/the-best-supercar-manufacturers-in-the-world-ranked/ss-AA1vneRe)
 The story of supercar manufacturing is one of passion and endless innovation. Besides their super stylish look, these are high-performance automobiles that bring along a thrill while driving. Here ...
 
 [Top 10 Super Car Manufacturers -